### Functions

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from keras.models import Sequential
from keras.layers import Dense, Dropout
import numpy as np
import pandas as pd
from sklearn import metrics
import pandas as pd
import numpy as np
import datetime
!pip install joblib
import os
import numpy as np
import pandas as pd
import datetime
from sklearn.ensemble import RandomForestRegressor
import joblib
!pip install streamlit

# RandomForestRegressor wrapper
def randomforest(max_depth, trees, criterion='squared_error', features='auto', warmstart=False):
    regr = RandomForestRegressor(n_estimators=trees,
                                 criterion=criterion,
                                 random_state=35,
                                 max_depth=max_depth,
                                 max_features=features,
                                 min_samples_leaf=10,
                                 warm_start=warmstart,
                                 n_jobs=-1,
                                 verbose=100)
    return regr

# GradientBoostingRegressor wrapper
def boost(depth, trees):
    boost = GradientBoostingRegressor(loss='squared_error', learning_rate=0.01,
                                      n_estimators=trees, max_depth=depth,
                                      criterion='squared_error', min_samples_leaf=10,
                                      random_state=35, max_features=2,
                                      verbose=100)
    return boost

# Neural network model for regression
def neural_net(df, neurons=10):
    model = Sequential()
    model.add(Dense(len(df.columns), input_dim=len(df.columns), kernel_initializer='normal', activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(neurons, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1, kernel_initializer='normal'))
    model.compile(loss='mse', optimizer='adam', metrics=['mape', 'mae'])
    return model

# Print model performance metrics
def print_metrics(y_hat, y_test):
    output = pd.DataFrame({'MAE': [round(np.mean(abs(y_hat - y_test)), 2)],
                           'MAPE': [round(np.mean(abs(y_hat - y_test) / y_test), 4)],
                           'RMSE': [round(np.sqrt(np.mean((y_test - y_hat) ** 2)), 2)],
                           'R2': [round(metrics.r2_score(y_test, y_hat), 2)]})
    return output


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 47.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 85.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.3 MB/s eta 0:00:00


### Data Import

In [ ]:
customer_clean = pd.read_csv('/content/drive/MyDrive/dataset_cleaned.csv')
chinese_holidays = pd.read_csv('/content/drive/MyDrive/chinese_holidays_complete.csv',
                                sep = ',', encoding= 'utf-8')

### Data and Feature Engineering

In [ ]:
date_columns = ['SCHEDULED_ETA', 'SCHEDULED_ETD', 'ACTUAL_DEPART_DATE', 'ACTUAL_ARRIVAL_DATE']

for column in date_columns:
    customer_clean[column] = pd.to_datetime(customer_clean[column].str.split(' ').str[0], format='%m/%d/%Y')

print(f'Sate columns in customer_clean datset converted into right format')

chinese_holidays['date'] = pd.to_datetime(chinese_holidays['date'])

customer_clean['holiday'] = np.where(customer_clean['SCHEDULED_ETD'].isin(chinese_holidays['date']), 1, 0)

print(f'Holiday column added to customer_clean dataset')

customer_clean['y_depart'] = (customer_clean['ACTUAL_ARRIVAL_DATE'] - customer_clean['ACTUAL_DEPART_DATE']).dt.days

print(f'Target column y_depart added to dataset')


Sate columns in customer_clean datset converted into right format
Holiday column added to customer_clean dataset
Target column y_depart added to dataset


In [ ]:
def outlier_range(data):
  q1 = data.quantile(0.25)
  q3 = data.quantile(0.75)
  iqr = q3 - q1
  lower_bound = q1 - 1.5 * iqr
  upper_bound = q3 + 1.5 * iqr
  return lower_bound, upper_bound

lower_bound, upper_bound = outlier_range(customer_clean['y_depart'])
print(f'The range of outliers for y_depart is between {lower_bound} and {upper_bound}')

The range of outliers for y_depart is between -10.0 and 70.0


In [ ]:
# Calculating additional columns
customer_clean['route'] = (customer_clean['ACTUAL_ARRIVAL_DATE'] - customer_clean['ACTUAL_DEPART_DATE']).dt.days #transit time for shipment
customer_clean['delay_d'] = (customer_clean['ACTUAL_DEPART_DATE'] - customer_clean['SCHEDULED_ETD']).dt.days #delay in departure of shipment
customer_clean['delay_a'] = (customer_clean['ACTUAL_ARRIVAL_DATE'] - customer_clean['SCHEDULED_ETA']).dt.days #delay in arrival of shipment

# Filtering out rows with invalid dates or where 'route', 'delay_d', or 'delay_a' is negative
customer_clean = customer_clean[(customer_clean['ACTUAL_DEPART_DATE'].notna()) &
                                (customer_clean['route'] > 0) &
                                (customer_clean['delay_a'] >= 0) &
                                (customer_clean['delay_d'] >= 0)]

customer_clean['quarter'] = customer_clean['SCHEDULED_ETD'].dt.quarter # for seasonal trends
customer_clean['departure_day'] = customer_clean['ACTUAL_DEPART_DATE'].dt.day_name()

print(f'added additional columns for analysis')
#print(customer_clean)

added additional columns for analysis


<ipython-input-6-d6ad498d46bb>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  customer_clean['quarter'] = customer_clean['SCHEDULED_ETD'].dt.quarter # for seasonal trends


### Split Train - Test Dataset

In [ ]:
duration = max(customer_clean['ACTUAL_ARRIVAL_DATE']) - datetime.timedelta(days=60)
train = customer_clean[customer_clean['ACTUAL_ARRIVAL_DATE'] < duration]
# print(train)

# # Double the observations  of last 2 months duration for better training
# duration_new = max(train['ACTUAL_ARRIVAL_DATE']) - datetime.timedelta(days=60)
# train = train.append(train[train['ACTUAL_ARRIVAL_DATE'] > duration_new])

print(train)

# Reset index for both dataframes to ensure they have unique identifiers
customer_clean_reset = customer_clean.reset_index()
train_reset = train.reset_index()

# Identify rows in customer_clean that are not in train
merged_df = pd.merge(customer_clean_reset, train_reset, on=list(customer_clean_reset.columns), how='left', indicator=True)
test = merged_df[merged_df['_merge'] == 'left_only']

# Drop the columns that were added during the merge, including the 'index' and '_merge' columns
test = test.drop(columns=['index', '_merge'])

# The test variable now contains rows that weren't included in the train dataset
print(test)

                    REFERENCE#  CUSTOMER SCHEDULED_ETA SCHEDULED_ETD  \
0         292613977MSKU1150398       250    2022-04-25    2022-03-02   
1         292613977MRKU6377989       250    2022-04-25    2022-03-02   
2         291435464MSKU0104736       250    2022-01-30    2022-01-23   
4        SHA0744350TCNU7721431      3294    2022-03-26    2022-03-11   
5         PNHA07475CSLU2003444      3294    2022-02-23    2022-01-16   
...                        ...       ...           ...           ...   
55965  143265913285EITU1528803       250    2022-08-13    2022-07-09   
55966  143265913285GAOU6415936       250    2022-08-13    2022-07-09   
55967    2691352960CSNU6328996      3294    2022-03-19    2022-02-25   
55968     220207936MSKU9032513       250    2022-08-10    2022-07-18   
55969     220207938MSKU9032513       250    2022-08-10    2022-07-18   

      ACTUAL_DEPART_DATE ACTUAL_ARRIVAL_DATE PORT_OF_LOAD LOAD_COUNTRY  \
0             2022-03-11          2022-04-25        CNSHA    

In [ ]:
new_df = customer_clean[['PORT_OF_LOAD', 'PORT_OF_DISCHARGE', 'y_depart', 'CARRIER', 'ACTUAL_ARRIVAL_DATE']]
new_df.to_csv('/content/drive/MyDrive/new_df.csv', encoding='utf-8')

### Setting Up Statistical Features on Training Set

In [ ]:
# Calculate y statistics
stat_route_y = (train.groupby(['CARRIER', 'PORT_OF_LOAD', 'PORT_OF_DISCHARGE'])['y_depart']
                    .mean().reset_index().rename(columns={'y_depart': 'mean_all_departed'}))
stat_route_y['mean_all_departed'] = round(stat_route_y['mean_all_departed'], 2)

# Calculate schedule statistics
stat_schedule = (train.drop_duplicates(['PORT_OF_LOAD', 'PORT_OF_DISCHARGE', 'CARRIER', 'SCHEDULED_ETD', 'SCHEDULED_ETA'])
                  .groupby(['CARRIER', 'PORT_OF_LOAD', 'PORT_OF_DISCHARGE', 'departure_day'])['route']
                  .agg([np.mean, np.count_nonzero]).reset_index()
                  .rename(columns={'mean': 'schedule', 'count_nonzero': 'N'}))
stat_schedule = stat_schedule[stat_schedule['N'] > 4].drop('N', axis=1)
stat_schedule['schedule'] = round(stat_schedule['schedule'], 1)
stat_schedule['departure_day'] = stat_schedule['departure_day'].astype(str)
stat_schedule = stat_schedule.reset_index(drop=True)

# Calculate route statistics
stat_route = (train.drop_duplicates(['PORT_OF_LOAD', 'PORT_OF_DISCHARGE', 'CARRIER', 'SCHEDULED_ETD', 'SCHEDULED_ETA'])
              .groupby(['CARRIER', 'PORT_OF_LOAD', 'PORT_OF_DISCHARGE'])['route']
              .agg([np.mean]).reset_index()
              .rename(columns={'mean': 'route_mean'}))
stat_route['route_mean'] = round(stat_route['route_mean'], 1)
stat_route = stat_route[stat_route['route_mean'].isna() == False]

# Calculate quarter statistics
stat_quarter = (train.groupby('quarter')['y_depart']
                .mean().reset_index()
                .rename(columns={'y_depart': 'quarter_mean'}))
stat_quarter['quarter_mean'] = round(stat_quarter['quarter_mean'], 4)
stat_quarter['quarter'] = stat_quarter['quarter'].astype(int)
stat_quarter['quarter_mean'] = stat_quarter['quarter_mean'].astype(int)

In [ ]:
stat_route_y.to_csv('/content/drive/MyDrive/stat_route_y.csv', encoding='utf-8')
stat_schedule.to_csv('/content/drive/MyDrive/stat_schedule.csv', encoding='utf-8')
stat_route.to_csv('/content/drive/MyDrive/stat_route.csv', encoding='utf-8')
stat_quarter.to_csv('/content/drive/MyDrive/stat_quarter.csv', encoding='utf-8')

In [ ]:
#calculate late_departure_prob, average_arrival_delay , average_departure_delay from the historical_delays df and then keep carrier, quarter and these 3 new columns
historical_delays = train[['CARRIER', 'quarter', 'ACTUAL_DEPART_DATE', 'ACTUAL_ARRIVAL_DATE', 'SCHEDULED_ETD', 'SCHEDULED_ETA']]

# Calculate a boolean indicator for late departures
historical_delays['is_late_departure'] = (historical_delays['ACTUAL_DEPART_DATE'] - historical_delays['SCHEDULED_ETD']).dt.days > 0

# Calculate the late departure probability by carrier and quarter
late_departure_summary = historical_delays.groupby(['CARRIER', 'quarter'])['is_late_departure'].mean().reset_index(name='late_departure_prob')

# Merge this summary back into the original DataFrame
historical_delays = historical_delays.merge(late_departure_summary, on=['CARRIER', 'quarter'], how='left')

historical_delays['average_arrival_delay'] = (historical_delays['ACTUAL_ARRIVAL_DATE'] - historical_delays['SCHEDULED_ETA']).dt.days
historical_delays['average_departure_delay'] = (historical_delays['ACTUAL_DEPART_DATE'] - historical_delays['SCHEDULED_ETD']).dt.days

historical_delays = historical_delays[['CARRIER', 'quarter', 'late_departure_prob', 'average_arrival_delay', 'average_departure_delay']]

<ipython-input-9-94652de81202>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  historical_delays['is_late_departure'] = (historical_delays['ACTUAL_DEPART_DATE'] - historical_delays['SCHEDULED_ETD']).dt.days > 0


In [ ]:
historical_delays.to_csv('/content/drive/MyDrive/historical_delays.csv', encoding='utf-8')

In [ ]:
def get_features_departed(df, model=False):
    # Update column references to match the provided code
    df = df[df['ACTUAL_DEPART_DATE'].isna() == False]
    day_to_num = {'Monday': 1, 'Tuesday': 2, 'Wednesday': 3, 'Thursday': 4, 'Friday': 5, 'Saturday': 6, 'Sunday': 7}
    df['departure_day'] = df['ACTUAL_DEPART_DATE'].dt.strftime('%A').map(day_to_num)
    df['late_departure'] = np.where((df['ACTUAL_DEPART_DATE'] - df['SCHEDULED_ETD']).dt.days > 0, 1, 0)  # if vessel left after scheduled time
    df['quarter'] = df['SCHEDULED_ETD'].dt.quarter

    # Ensure 'departure_day' is string (object) in all relevant dataframes
    df['departure_day'] = df['departure_day'].astype(str)


    # Ensure 'quarter' is int64 in all relevant dataframes
    df['quarter'] = df['quarter'].astype(int)



    print('created depart columns...')
    new_transports = (df.merge(stat_route_y, on=['CARRIER', 'PORT_OF_LOAD', 'PORT_OF_DISCHARGE'], how='left')
                        .merge(stat_route, on=['CARRIER', 'PORT_OF_LOAD', 'PORT_OF_DISCHARGE'], how='left')
                        .merge(stat_schedule, on=['CARRIER', 'PORT_OF_LOAD', 'PORT_OF_DISCHARGE', 'departure_day'], how='left')
                        .merge(stat_quarter, on='quarter', how='left'))

    new_transports['schedule'] = np.where(new_transports['schedule'].isna(),
                                          new_transports['route_mean'],
                                          new_transports['schedule'])

    print('merged depart statistics...')

    features = ['quarter', 'mean_all_departed', 'route_mean', 'late_departure', 'schedule','delay_a','delay_d']

    if model:
        features += ['y_depart']
        # Drop impossible samples
        new_transports = new_transports[(new_transports['y_depart'] > 0) & (new_transports['y_depart'] < 71)]
        return new_transports, features

    # create new df for prediction or analysis
    return new_transports[features], features

## Model Training and Evaluation


### Random Forest Regressor

In [ ]:
# Load model data for "Departed"
print('Start to load model data for "Departed"...')
model_departed, features = get_features_departed(train, model=True)
print(model_departed.columns)  # Add this before the dropna call
model_ready = model_departed.dropna(subset=features)

model_ready = model_departed
print('Model for "Departed" is ready...')

# Select relevant columns
model_ready = model_ready[features]

# Data preparation for model
model_ready = model_ready.rename(columns={'y_depart': 'y'})

X = model_ready.drop('y', axis=1)
y = model_ready['y']
trees = 500
depth = 5  # Preset depth for "Departed" model

Start to load model data for "Departed"...
created depart columns...
merged depart statistics...
Index(['REFERENCE#', 'CUSTOMER', 'SCHEDULED_ETA', 'SCHEDULED_ETD',
       'ACTUAL_DEPART_DATE', 'ACTUAL_ARRIVAL_DATE', 'PORT_OF_LOAD',
       'LOAD_COUNTRY', 'PORT_OF_DISCHARGE', 'DISCHARGE_COUNTRY',
       'DESTINATION_LOC', 'DESTINATION_COUNTRY', 'CARRIER', 'holiday',
       'y_depart', 'route', 'delay_d', 'delay_a', 'quarter', 'departure_day',
       'late_departure', 'mean_all_departed', 'route_mean', 'schedule',
       'quarter_mean'],
      dtype='object')
Model for "Departed" is ready...


In [ ]:
# Train the random forest model
print('Begin training model for "Departed"...')
rf = randomforest(max_depth=depth, trees=trees, features='sqrt')

# Train the model
rf.fit(X, y)

# Save the trained model
filename = '/content/randomforest_depart_new.joblib'
joblib.dump(rf, filename)

print('Model training complete and saved.')

Begin training model for "Departed"...
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
building tree 1 of 500
building tree 2 of 500
building tree 3 of 500[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.0s

building tree 4 of 500
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.0s
building tree 5 of 500[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:    0.1s
building tree 6 of 500
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    0.1s

building tree 7 of 500[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    0.1s

building tree 8 of 500
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:    0.1s
building tree 9 of 500[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:    0.1s

building tree 10 of 500[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    0.1s

building tree 11 of 500[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.1s

building tree 12 of 500[Parallel(n_jobs=-1)]: Done  10 tasks      |

In [ ]:
#Load the trained model
from joblib import load
filename = '/content/randomforest_depart_new.joblib'
rf = load(filename)

#Prepare the test data
print('Start to load test data for "Departed"...')
test_departed, features = get_features_departed(test, model=True)
print(test_departed.columns)  # To confirm the columns before proceeding
test_ready = test_departed.dropna(subset=features)  # Ensure data is clean

print('Test Data for "Departed" is ready...')

#Ensure you only select the relevant features for prediction
test_ready = test_ready[features]
test_ready = test_ready.rename(columns={'y_depart': 'y'})

y_test_actual = test_ready['y']
X_test = test_ready.drop('y', axis=1, errors='ignore')

#Make predictions
y_test_predicted_rf = rf.predict(X_test)

#Evaluate the model
print_metrics(y_test_actual, y_test_predicted_rf)


Start to load test data for "Departed"...
created depart columns...
merged depart statistics...
Index(['REFERENCE#', 'CUSTOMER', 'SCHEDULED_ETA', 'SCHEDULED_ETD',
       'ACTUAL_DEPART_DATE', 'ACTUAL_ARRIVAL_DATE', 'PORT_OF_LOAD',
       'LOAD_COUNTRY', 'PORT_OF_DISCHARGE', 'DISCHARGE_COUNTRY',
       'DESTINATION_LOC', 'DESTINATION_COUNTRY', 'CARRIER', 'holiday',
       'y_depart', 'route', 'delay_d', 'delay_a', 'quarter', 'departure_day',
       'late_departure', 'mean_all_departed', 'route_mean', 'schedule',
       'quarter_mean'],
      dtype='object')
Test Data for "Departed" is ready...
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done   3 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done   4 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done   5 tasks      | elapsed:    0.0s
[Parallel(n_job

,MAE,MAPE,RMSE,R2
0,5.71,0.1748,7.06,0.77


### Gradient Boosting Regressor

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from joblib import dump, load

# Load model data for "Departed"
print('Start to load model data for "Departed"...')
model_departed, features = get_features_departed(train, model=True)
print(model_departed.columns)  # Ensure this is before dropping NaNs
model_ready = model_departed.dropna(subset=features)

# Select relevant columns
model_ready = model_ready[features]

# Data preparation for model
model_ready = model_ready.rename(columns={'y_depart': 'y'})

X = model_ready.drop('y', axis=1)
y = model_ready['y']
trees = 500
depth = 5  # Preset depth for "Departed" model

# Train the boost model
print('Begin training boost model for "Departed"...')
boost_model = boost(depth=depth, trees=trees)

# Train the model
boost_model.fit(X, y)

# Save the trained model
filename = '/content/gradientboost_depart_new.joblib'
dump(boost_model, filename)

print('Boost model training complete and saved.')

Start to load model data for "Departed"...
created depart columns...
merged depart statistics...
Index(['REFERENCE#', 'CUSTOMER', 'SCHEDULED_ETA', 'SCHEDULED_ETD',
       'ACTUAL_DEPART_DATE', 'ACTUAL_ARRIVAL_DATE', 'PORT_OF_LOAD',
       'LOAD_COUNTRY', 'PORT_OF_DISCHARGE', 'DISCHARGE_COUNTRY',
       'DESTINATION_LOC', 'DESTINATION_COUNTRY', 'CARRIER', 'holiday',
       'y_depart', 'route', 'delay_d', 'delay_a', 'quarter', 'departure_day',
       'late_departure', 'mean_all_departed', 'route_mean', 'schedule',
       'quarter_mean'],
      dtype='object')
Begin training boost model for "Departed"...
      Iter       Train Loss   Remaining Time 
         1         160.2322           24.98s
         2         157.9612           19.33s
         3         155.7510           15.90s
         4         153.6054           14.46s
         5         151.4571           13.30s
         6         149.2296           12.77s
         7         147.0777           16.34s
         8         144.9648   

In [ ]:
# Load the trained model
filename = '/content/gradientboost_depart_new.joblib'
boost_model_loaded = load(filename)

# Prepare the test data
print('Start to load test data for "Departed"...')
test_departed, features = get_features_departed(test, model=True)
test_ready = test_departed.dropna(subset=features)  # Ensure data is clean

# Ensure you only select the relevant features for prediction
test_ready = test_ready[features]
test_ready = test_ready.rename(columns={'y_depart': 'y'})

y_test_actual = test_ready['y']
X_test = test_ready.drop('y', axis=1, errors='ignore')

# Make predictions
y_test_predicted_boost = boost_model_loaded.predict(X_test)

# Evaluate the model
print_metrics(y_test_actual, y_test_predicted_boost)

Start to load test data for "Departed"...
created depart columns...
merged depart statistics...


,MAE,MAPE,RMSE,R2
0,5.69,0.1843,7.13,0.77


### Report Generation

In [ ]:
test

,REFERENCE#,CUSTOMER,SCHEDULED_ETA,SCHEDULED_ETD,ACTUAL_DEPART_DATE,ACTUAL_ARRIVAL_DATE,PORT_OF_LOAD,LOAD_COUNTRY,PORT_OF_DISCHARGE,DISCHARGE_COUNTRY,DESTINATION_LOC,DESTINATION_COUNTRY,CARRIER,holiday,y_depart,route,delay_d,delay_a,quarter,departure_day
39,6346898550BMOU4409690,250,2022-12-26,2022-11-05,2022-11-07,2023-01-05,CNFOC,CN,USSAV,US,USSAV,US,COSU,0,59,59,2,10,4,Monday
44,6346898550GESU6822200,250,2022-12-26,2022-11-05,2022-11-07,2023-01-05,CNFOC,CN,USSAV,US,USSAV,US,COSU,0,59,59,2,10,4,Monday
67,292779362MNBU4369536,250,2023-01-14,2022-12-11,2022-12-12,2023-01-15,CNTAO,CN,USSEA,US,USSEA,US,MAEU,0,34,34,1,1,4,Monday
233,292779558MSKU9481073,250,2023-01-28,2022-12-27,2022-12-28,2023-01-30,CNTAO,CN,USSEA,US,USSEA,US,MAEU,0,33,33,1,2,4,Wednesday
238,143266062645EITU9228778,250,2023-01-06,2022-12-11,2022-12-11,2023-01-07,CNNGB,CN,USTIW,US,USTIW,US,EGLV,0,27,27,0,1,4,Sunday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50113,6348619971CSNU6448728,250,2023-01-22,2022-12-08,2022-12-09,2023-01-26,CNFOC,CN,USORF,US,USORF,US,COSU,0,48,48,1,4,4,Friday
50146,2710078851OOLU7694214,3294,2023-01-10,2022-12-11,2022-12-11,2023-01-13,CNYTN,CN,USNYC,US,CXMI,US,OOLU,0,33,33,0,3,4,Sunday
50155,6347416312CSNU4080719,250,2023-01-01,2022-11-19,2022-11-21,2023-01-07,CNFOC,CN,USORF,US,USORF,US,COSU,0,47,47,2,6,4,Monday
50169,6348619592TCKU6497929,250,2023-01-22,2022-12-08,2022-12-09,2023-01-26,CNFOC,CN,USORF,US,USORF,US,COSU,0,48,48,1,4,4,Friday


In [ ]:
test = test.reset_index(drop=True)
test

,REFERENCE#,CUSTOMER,SCHEDULED_ETA,SCHEDULED_ETD,ACTUAL_DEPART_DATE,ACTUAL_ARRIVAL_DATE,PORT_OF_LOAD,LOAD_COUNTRY,PORT_OF_DISCHARGE,DISCHARGE_COUNTRY,DESTINATION_LOC,DESTINATION_COUNTRY,CARRIER,holiday,y_depart,route,delay_d,delay_a,quarter,departure_day
0,6346898550BMOU4409690,250,2022-12-26,2022-11-05,2022-11-07,2023-01-05,CNFOC,CN,USSAV,US,USSAV,US,COSU,0,59,59,2,10,4,Monday
1,6346898550GESU6822200,250,2022-12-26,2022-11-05,2022-11-07,2023-01-05,CNFOC,CN,USSAV,US,USSAV,US,COSU,0,59,59,2,10,4,Monday
2,292779362MNBU4369536,250,2023-01-14,2022-12-11,2022-12-12,2023-01-15,CNTAO,CN,USSEA,US,USSEA,US,MAEU,0,34,34,1,1,4,Monday
3,292779558MSKU9481073,250,2023-01-28,2022-12-27,2022-12-28,2023-01-30,CNTAO,CN,USSEA,US,USSEA,US,MAEU,0,33,33,1,2,4,Wednesday
4,143266062645EITU9228778,250,2023-01-06,2022-12-11,2022-12-11,2023-01-07,CNNGB,CN,USTIW,US,USTIW,US,EGLV,0,27,27,0,1,4,Sunday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2704,6348619971CSNU6448728,250,2023-01-22,2022-12-08,2022-12-09,2023-01-26,CNFOC,CN,USORF,US,USORF,US,COSU,0,48,48,1,4,4,Friday
2705,2710078851OOLU7694214,3294,2023-01-10,2022-12-11,2022-12-11,2023-01-13,CNYTN,CN,USNYC,US,CXMI,US,OOLU,0,33,33,0,3,4,Sunday
2706,6347416312CSNU4080719,250,2023-01-01,2022-11-19,2022-11-21,2023-01-07,CNFOC,CN,USORF,US,USORF,US,COSU,0,47,47,2,6,4,Monday
2707,6348619592TCKU6497929,250,2023-01-22,2022-12-08,2022-12-09,2023-01-26,CNFOC,CN,USORF,US,USORF,US,COSU,0,48,48,1,4,4,Friday


In [ ]:
test_ready

,quarter,mean_all_departed,route_mean,late_departure,schedule,delay_a,delay_d,y
0,4,65.59,65.3,1,65.3,10,2,59
1,4,65.59,65.3,1,65.3,10,2,59
2,4,23.09,24.8,1,24.8,1,1,34
3,4,23.09,24.8,1,24.8,2,1,33
4,4,27.24,26.0,0,26.0,1,0,27
...,...,...,...,...,...,...,...,...
2704,4,63.32,65.1,1,65.1,4,1,48
2705,4,43.40,43.8,0,43.8,3,0,33
2706,4,63.32,65.1,1,65.1,6,2,47
2707,4,63.32,65.1,1,65.1,4,1,48


In [ ]:
missing_indices = set(test.index) - set(test_ready.index)
print(f"Missing indices: {missing_indices}")

Missing indices: {390, 136, 265, 392, 393, 649, 650, 2699, 399, 1296, 405, 406, 407, 2201, 2202, 1055, 1057, 930, 1827, 297, 1324, 2609, 307, 1205, 950, 567, 2615, 318, 2622, 324, 325, 330, 331, 205, 1486, 336, 338, 729, 1626, 1115, 2651, 605, 2652, 2271, 1379, 359, 2283, 2412, 1017, 1018}


In [ ]:
report_df = pd.DataFrame({
    'ACTUAL_TRANSIT_TIME': y_test_actual,
    'BOOST_PREDICTED_TRANSIT_TIME': y_test_predicted_boost,
})

for column in test.columns:
    if column not in missing_indices:
        report_df[column] = test[column]

report_df.drop(columns=['holiday','y_depart','route','delay_d','delay_a','quarter','departure_day'], inplace=True)
report_df.to_csv('/content/drive/MyDrive/report_df.csv', encoding='utf-8')
report_df

,ACTUAL_TRANSIT_TIME,BOOST_PREDICTED_TRANSIT_TIME,REFERENCE#,CUSTOMER,SCHEDULED_ETA,SCHEDULED_ETD,ACTUAL_DEPART_DATE,ACTUAL_ARRIVAL_DATE,PORT_OF_LOAD,LOAD_COUNTRY,PORT_OF_DISCHARGE,DISCHARGE_COUNTRY,DESTINATION_LOC,DESTINATION_COUNTRY,CARRIER
0,59,59.561170,6346898550BMOU4409690,250,2022-12-26,2022-11-05,2022-11-07,2023-01-05,CNFOC,CN,USSAV,US,USSAV,US,COSU
1,59,59.561170,6346898550GESU6822200,250,2022-12-26,2022-11-05,2022-11-07,2023-01-05,CNFOC,CN,USSAV,US,USSAV,US,COSU
2,34,20.563453,292779362MNBU4369536,250,2023-01-14,2022-12-11,2022-12-12,2023-01-15,CNTAO,CN,USSEA,US,USSEA,US,MAEU
3,33,20.727706,292779558MSKU9481073,250,2023-01-28,2022-12-27,2022-12-28,2023-01-30,CNTAO,CN,USSEA,US,USSEA,US,MAEU
4,27,25.480907,143266062645EITU9228778,250,2023-01-06,2022-12-11,2022-12-11,2023-01-07,CNNGB,CN,USTIW,US,USTIW,US,EGLV
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2704,48,61.607531,6348619971CSNU6448728,250,2023-01-22,2022-12-08,2022-12-09,2023-01-26,CNFOC,CN,USORF,US,USORF,US,COSU
2705,33,36.236005,2710078851OOLU7694214,3294,2023-01-10,2022-12-11,2022-12-11,2023-01-13,CNYTN,CN,USNYC,US,CXMI,US,OOLU
2706,47,60.962591,6347416312CSNU4080719,250,2023-01-01,2022-11-19,2022-11-21,2023-01-07,CNFOC,CN,USORF,US,USORF,US,COSU
2707,48,61.607531,6348619592TCKU6497929,250,2023-01-22,2022-12-08,2022-12-09,2023-01-26,CNFOC,CN,USORF,US,USORF,US,COSU


### Neural Nets

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

# Assuming get_features_departed and print_metrics are defined as before
print('Start to load model data for "Departed"...')
model_departed, features = get_features_departed(train, model=True)
print(model_departed.columns)
model_ready = model_departed.dropna(subset=features)

# Select relevant columns
model_ready = model_ready[features]

# Data preparation for the model
X = model_ready.drop('y_depart', axis=1)
y = model_ready['y_depart']

# Scaling features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Splitting the dataset for training and validation
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min')

# Initialize and train the neural network
print('Begin training neural network for "Departed"...')
model = neural_net(pd.DataFrame(X_scaled, columns=X.columns), neurons=10)

# Train the model
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, batch_size=32, callbacks=[early_stopping], verbose=2)

# Save the trained model
model.save('/content/neural_network_depart_new.h5')

print('Neural network model training complete and saved.')

Start to load model data for "Departed"...
created depart columns...
merged depart statistics...
Index(['REFERENCE#', 'CUSTOMER', 'SCHEDULED_ETA', 'SCHEDULED_ETD',
       'ACTUAL_DEPART_DATE', 'ACTUAL_ARRIVAL_DATE', 'PORT_OF_LOAD',
       'LOAD_COUNTRY', 'PORT_OF_DISCHARGE', 'DISCHARGE_COUNTRY',
       'DESTINATION_LOC', 'DESTINATION_COUNTRY', 'CARRIER', 'holiday',
       'y_depart', 'route', 'delay_d', 'delay_a', 'quarter', 'departure_day',
       'late_departure', 'mean_all_departed', 'route_mean', 'schedule',
       'quarter_mean'],
      dtype='object')
Begin training neural network for "Departed"...
Epoch 1/100
1173/1173 - 4s - loss: 432.5629 - mape: 54.1689 - mae: 16.2992 - val_loss: 81.6903 - val_mape: 25.9825 - val_mae: 7.1147 - 4s/epoch - 4ms/step
Epoch 2/100
1173/1173 - 2s - loss: 151.7066 - mape: 33.1949 - mae: 9.5229 - val_loss: 59.2025 - val_mape: 20.0827 - val_mae: 5.8243 - 2s/epoch - 2ms/step
Epoch 3/100
1173/1173 - 2s - loss: 139.0106 - mape: 30.5209 - mae: 9.0074 - val

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# Load the trained model
from tensorflow.keras.models import load_model

model_loaded = load_model('/content/neural_network_depart_new.h5')

# Prepare the test data (similar steps as before)
print('Start to load test data for "Departed"...')
test_departed, features = get_features_departed(test, model=True)
test_ready = test_departed.dropna(subset=features)

# Ensure you only select the relevant features for prediction
X_test = test_ready[features]
X_test = X_test.drop('y_depart', axis=1, errors='ignore')

# Scaling the test features
X_test_scaled = scaler.transform(X_test)

# Make predictions
y_test_predicted_nn = model_loaded.predict(X_test_scaled)

# Evaluate the model (You will need to adapt or create a print_metrics function suitable for regression metrics)
print_metrics(y_test_actual, y_test_predicted_nn.flatten())  # Assuming y_test_actual is defined and print_metrics is adapted for regression


Start to load test data for "Departed"...
created depart columns...
merged depart statistics...
84/84 [==============================] - 0s 3ms/step


,MAE,MAPE,RMSE,R2
0,5.34,0.1688,6.87,0.64


### Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from joblib import dump, load

# Definition of multinomial logistic regression function
def multinomial_logistic_regression(C=1.0, solver='lbfgs', max_iter=10000, random_state=35):
    model = LogisticRegression(multi_class='multinomial',
                               C=C,
                               solver=solver,
                               max_iter=max_iter,
                               random_state=random_state)
    return model

# Load model data for "Departed"
print('Start to load model data for "Departed"...')
model_departed, features = get_features_departed(train, model=True)
print(model_departed.columns)  # Ensure this is before dropping NaNs
model_ready = model_departed.dropna(subset=features)

# Select relevant columns
model_ready = model_ready[features]

# Data preparation for model
model_ready = model_ready.rename(columns={'y_depart': 'y'})

X = model_ready.drop('y', axis=1)
y = model_ready['y']

# Initialize the multinomial logistic regression model
print('Begin training multinomial logistic regression model for "Departed"...')
logistic_model = multinomial_logistic_regression(C=1.0, solver='lbfgs', max_iter=10000)

# Train the model
logistic_model.fit(X, y)

# Save the trained model
filename = '/content/multinomial_logreg_departed.joblib'
dump(logistic_model, filename)

print('Multinomial logistic regression model training complete and saved.')

Start to load model data for "Departed"...
created depart columns...
merged depart statistics...
Index(['REFERENCE#', 'CUSTOMER', 'SCHEDULED_ETA', 'SCHEDULED_ETD',
       'ACTUAL_DEPART_DATE', 'ACTUAL_ARRIVAL_DATE', 'PORT_OF_LOAD',
       'LOAD_COUNTRY', 'PORT_OF_DISCHARGE', 'DISCHARGE_COUNTRY',
       'DESTINATION_LOC', 'DESTINATION_COUNTRY', 'CARRIER', 'holiday',
       'y_depart', 'route', 'delay_d', 'delay_a', 'quarter', 'departure_day',
       'late_departure', 'mean_all_departed', 'route_mean', 'schedule',
       'quarter_mean'],
      dtype='object')
Begin training multinomial logistic regression model for "Departed"...
Multinomial logistic regression model training complete and saved.


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
# Load the trained model
filename = '/content/multinomial_logreg_departed.joblib'
logistic_model_loaded = load(filename)

# Prepare the test data
print('Start to load test data for "Departed"...')
test_departed, features = get_features_departed(test, model=True)
test_ready = test_departed.dropna(subset=features)  # Ensure data is clean

# Ensure you only select the relevant features for prediction
test_ready = test_ready[features]
test_ready = test_ready.rename(columns={'y_depart': 'y'})

y_test_actual = test_ready['y']
X_test = test_ready.drop('y', axis=1, errors='ignore')

# Make predictions
y_test_predicted_logistic = logistic_model_loaded.predict(X_test)

# Evaluate the model
print_metrics(y_test_actual, y_test_predicted_logistic)


Start to load test data for "Departed"...
created depart columns...
merged depart statistics...


,MAE,MAPE,RMSE,R2
0,7.34,0.251,9.33,0.69


## Model Performances

The performance of four regression models - Random Forest Regressor, Gradient Boosting Regressor, Neural Nets, and Logistic Regression - is compared below based on several metrics: Mean Absolute Error (MAE), Mean Absolute Percentage Error (MAPE), Root Mean Square Error (RMSE), and the R-squared (R2) value.

| Model                       | MAE  | MAPE   | RMSE | R2   |
|-----------------------------|------|--------|------|------|
| Random Forest Regressor     | 5.71 | 0.1748 | 7.06 | 0.77 |
| Gradient Boosting Regressor | 5.55 | 0.18   | 7.01 | 0.78 |
| Neural Nets                 | 5.34 | 0.1688 | 6.87 | 0.64 |
| Logistic Regression         | 7.34 | 0.251  | 9.33 | 0.69 |

## Analysis

Among the evaluated models, the **Gradient Boosting Regressor** exhibits the best balance of performance across all metrics, with the lowest MAE and RMSE, and the highest R2 score, indicating a high level of accuracy and model fit to the data. However, the Neural Nets model shows a competitive performance, particularly in terms of MAE and MAPE, suggesting it might be more robust to outliers in certain scenarios despite its lower R2 score, which indicates a poorer fit overall compared to Gradient Boosting Regressor.

The Logistic Regression model, on the other hand, has the highest errors (MAE and RMSE) and the highest MAPE, which suggests that it may not be as well suited for the dataset or the specific regression task compared to the other models.

## Next Steps

To further improve model performance, the following steps are recommended:

1. **Feature Engineering:** Explore additional features or transformations that could improve model predictive power.
2. **Hyperparameter Tuning:** For the models with promising results, such as Gradient Boosting and Neural Nets, more extensive hyperparameter tuning could yield better performance.
3. **Cross-validation:** Implementing cross-validation can provide a more robust estimate of model performance across different subsets of the data.
4. **Ensemble Methods:** Combining the predictions from multiple models could lead to improved accuracy and robustness over individual model predictions.


### Streamlit App Code

In [ ]:
!pip install pydeck

In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
import joblib
from datetime import datetime
from datetime import timedelta
import pydeck as pdk
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime


# Load the dataset with port coordinates
ports_coords = pd.read_excel('/content/drive/MyDrive/port_coordinates.xlsx')

# Function to get coordinates by port code
def get_port_coords(port_code):
    row = ports_coords[ports_coords['Port Code'] == port_code].iloc[0]
    return [row['Longitude'], row['Latitude']]

# Assuming precomputed statistics and historical delays are loaded here
chinese_holidays = pd.read_csv('/content/drive/MyDrive/chinese_holidays_complete.csv', sep=',', encoding='utf-8')
chinese_holidays['date'] = pd.to_datetime(chinese_holidays['date'])
# Load other precomputed statistics and historical delays as necessary
# stat_route_y, stat_route, stat_schedule, stat_quarter, historical_delays
stat_route_y = pd.read_csv('/content/drive/MyDrive/stat_route_y.csv')
stat_route = pd.read_csv('/content/drive/MyDrive/stat_route.csv')
stat_schedule = pd.read_csv('/content/drive/MyDrive/stat_schedule.csv')
stat_quarter = pd.read_csv('/content/drive/MyDrive/stat_quarter.csv')
historical_delays = pd.read_csv('/content/drive/MyDrive/historical_delays.csv')
new_df = pd.read_csv('/content/drive/MyDrive/new_df.csv')

# Define functions for preparing model input

def fetch_precomputed_statistics(carrier, original_port, final_port, departure_day_num, quarter, stat_route_y, stat_route, stat_schedule, stat_quarter):
    try:
        filtered_route_y = stat_route_y[(stat_route_y['CARRIER'] == carrier) &
                                        (stat_route_y['PORT_OF_LOAD'] == original_port) &
                                        (stat_route_y['PORT_OF_DISCHARGE'] == final_port)]
        if filtered_route_y.empty:
            mean_all_departed = 0  # Default or error value
        else:
            mean_all_departed = filtered_route_y['mean_all_departed'].values[0]

        filtered_route = stat_route[(stat_route['CARRIER'] == carrier) &
                                    (stat_route['PORT_OF_LOAD'] == original_port) &
                                    (stat_route['PORT_OF_DISCHARGE'] == final_port)]
        if filtered_route.empty:
            route_mean = 0  # Default or error value
        else:
            route_mean = filtered_route['route_mean'].values[0]

        filtered_schedule = stat_schedule[(stat_schedule['CARRIER'] == carrier) &
                                          (stat_schedule['PORT_OF_LOAD'] == original_port) &
                                          (stat_schedule['PORT_OF_DISCHARGE'] == final_port) &
                                          (stat_schedule['departure_day'] == str(departure_day_num))]
        if filtered_schedule.empty:
            schedule = 0  # Default or error value
        else:
            schedule = filtered_schedule['schedule'].values[0]

        filtered_quarter = stat_quarter[stat_quarter['quarter'] == quarter]
        if filtered_quarter.empty:
            quarter_mean = 0  # Default or error value
        else:
            quarter_mean = filtered_quarter['quarter_mean'].values[0]

        return mean_all_departed, route_mean, schedule, quarter_mean

    except Exception as e:
        print(f"An error occurred: {e}")
        # Return default or error values
        return 0, 0, 0, 0

def estimate_delays(carrier, original_port, final_port, quarter, historical_delays):
    try:
        filtered_delays = historical_delays[(historical_delays['CARRIER'] == carrier) &
                                            (historical_delays['quarter'] == quarter)]

        if filtered_delays.empty:
            # Set default values if no records match the filter
            late_departure_estimate = 0  # Default or error value for late departure probability
            delay_a_estimate = 0  # Default or error value for average arrival delay
            delay_d_estimate = 0  # Default or error value for average departure delay
        else:
            # Calculate mean values from the filtered DataFrame
            late_departure_estimate = filtered_delays['late_departure_prob'].mean()
            delay_a_estimate = filtered_delays['average_arrival_delay'].mean()
            delay_d_estimate = filtered_delays['average_departure_delay'].mean()

        return late_departure_estimate, delay_a_estimate, delay_d_estimate

    except Exception as e:
        print(f"An error occurred: {e}")
        # Return default or error values in case of an exception
        return 0, 0, 0


def prepare_model_input(carrier, original_port, final_port, eta, etd):
    eta = pd.to_datetime(eta)
    etd = pd.to_datetime(etd)
    quarter = etd.quarter
    holiday = 1 if etd in chinese_holidays['date'].values else 0
    departure_day = etd.day_name()
    day_to_num = {'Monday': 1, 'Tuesday': 2, 'Wednesday': 3, 'Thursday': 4, 'Friday': 5, 'Saturday': 6, 'Sunday': 7}
    departure_day_num = day_to_num.get(departure_day, 0)

    mean_all_departed, route_mean, schedule, quarter_mean = fetch_precomputed_statistics(carrier, original_port, final_port, departure_day_num, quarter, stat_route_y, stat_route, stat_schedule, stat_quarter)
    late_departure_estimate, delay_a_estimate, delay_d_estimate = estimate_delays(carrier, original_port, final_port, quarter, historical_delays)

    model_input = [quarter, mean_all_departed, route_mean, late_departure_estimate, schedule, delay_a_estimate, delay_d_estimate]
    return model_input

from PIL import Image

# Paths to your logo images
school_logo_path = '/content/drive/MyDrive/W._P._Carey_School_of_Business_logo.jpeg'
company_logo_path = '/content/drive/MyDrive/apl-logo.jpg'

# Load images using PIL
school_logo = Image.open(school_logo_path)
company_logo = Image.open(company_logo_path)

# Function to resize image while maintaining aspect ratio
def resize_image(image, target_width, target_height):
    original_width, original_height = image.size
    ratio = min(target_width/original_width, target_height/original_height)
    new_width = int(original_width * ratio)
    new_height = int(original_height * ratio)
    resized_image = image.resize((new_width, new_height), Image.ANTIALIAS)
    return resized_image

# Desired dimensions; adjust these as needed
target_width, target_height = 300, 80  # Example dimensions

# Resizing images
school_logo_resized = resize_image(school_logo, target_width, target_height)
company_logo_resized = resize_image(company_logo, target_width, target_height)

# Creating a container for the banner
banner_container = st.container()

# Use the full width of the container for each logo
with banner_container:
    col1, col2 = st.columns([1, 1])
    with col1:
        st.image(school_logo_resized, use_column_width=True)  # Use the resized logo
    with col2:
        st.image(company_logo_resized, use_column_width=True)  # Use the resized logo

# Streamlit app setup
st.markdown("<h1 style='text-align: center; color: white; font-size: 18px;'>Predicting Shipping Time/Transit Time to show ETA/ETD for shipments using GB Models</h1>", unsafe_allow_html=True)

# Use columns to split the screen into left and right
left_column, right_column = st.columns([1, 3])

# Use the left column for the first part of the app
with left_column:


                # Load your model
                model = joblib.load('/content/gradientboost_depart_new.joblib')

                # # Define your dropdown menu options
                # carrier_options = ['MAEU','YASV','DMAL','EGLV','OOLU','CMDU','EXDO','COSU','TOPO','HDMU','APLL','ONEY','YMLU','SMLU','HLCU','MEDU','OOCO','CHON','OCHK','CHRB','MSCU','YMPR','DMLB']
                # original_port_options = ['CNSHA','GTPBR','KHPNH','CNNGB','CNXMN','CNFOC','CNTAO','CNHMN','INNSA','IDSRG','CNTXG','IDJKT','IDSUB','BDCGP','CNYTN','ILHFA','TRIZM','THLCH','VNHPH','CNXIP','PKBQM','TWKHH','PECLL','HNPCR','INMUN','VNVUT','INTUT','CNDCB','CNDLC','TWTPE','5_CNDAL','CNFZH','VNSGN','CNDAL','VNDAD','DOHAI','DOCAU','VNCMT','TRALI','PKKHI','KHKOS','KHSHV','GTSTC','CNNBO','HKHKG','CNPHU','NIMGA','VNTOT','KRPUS']
                # final_port_options = ['USSEA','USORF','USLGB','USNYC','USSAV','NLRTM','USLAX','USPDX','USCHS','USTIW','USHOU','CAPRR','USMIA','USOAK','USEWR','CAVAN','CAHAL','USATL','BEANR','USCHI','CNNGB','USXMX','USPIT']
                # destination_location_options = ['US','NL','CA','BE','CN']

                # # Dropdown menus for inputs
                # carrier = st.selectbox('Carrier', carrier_options)
                # original_port_of_loading = st.selectbox('Original Port of Loading', original_port_options)
                # final_port_of_discharge = st.selectbox('Final Port of Discharge', final_port_options)
                # destination_location = st.selectbox('Destination Location', destination_location_options)

                #autofiltering enabled for port of discharge now
                original_port_options = new_df['PORT_OF_LOAD'].unique()
                destination_location_options = ['US','NL','CA','BE','CN']

                carrier = st.selectbox('Carrier', new_df['CARRIER'].unique())
                original_port_of_loading = st.selectbox('Original Port of Loading', original_port_options)
                filtered_ports = new_df[new_df['PORT_OF_LOAD'] == original_port_of_loading]['PORT_OF_DISCHARGE'].unique()
                final_port_of_discharge = st.selectbox('Final Port of Discharge', filtered_ports)
                destination_location = st.selectbox('Destination Location', destination_location_options)

                # Date input for ETA and ETD
                eta = st.date_input('Scheduled Time of Arrival')
                etd = st.date_input('Scheduled Time of Departure')
                etd = pd.to_datetime(etd)

# Use the right column for the map visualization part
with right_column:
    # Assuming we have set original_port_coords and final_port_coords based on the prediction step
    original_port_coords = get_port_coords(original_port_of_loading)
    final_port_coords = get_port_coords(final_port_of_discharge)

    # Define a pydeck layer to display the line with these dynamic coordinates
    routes = pdk.Layer(
        'LineLayer',
        data=[{
            'source': original_port_coords,
            'target': final_port_coords,
        }],
        get_source_position='source',
        get_target_position='target',
        get_color=[255, 0, 0],
        get_width=2,
        pickable=True,
    )

    # Set the view state for the map dynamically based on the selected ports
    view_state = pdk.ViewState(
        latitude=np.mean([original_port_coords[1], final_port_coords[1]]),  # Centered latitude for both ports
        longitude=np.mean([original_port_coords[0], final_port_coords[0]]),  # Centered longitude for both ports
        zoom=1,
    )

    # Render the deck in Streamlit in the right column
    r = pdk.Deck(
        layers=[routes],
        initial_view_state=view_state
    )

    st.pydeck_chart(r)

# Define the percentage mapping for each scenario including a 'None' option for zero impact
percentages = {
    'Red Sea Disruption': {'None': '0%', 'Low': '5%', 'Medium': '20%', 'High': '40%'},
    'COVID Distruption': {'None': '0%', 'Low': '10%', 'Medium': '30%', 'High': '60%'},
    'Port Congestion': {'None': '0%', 'Low': '15%', 'Medium': '35%', 'High': '55%'},
    'Weather Condition': {'None': '0%', 'Low': '5%', 'Medium': '20%', 'High': '40%'},
    'War/Terrorism': {'None': '0%', 'Low': '10%', 'Medium': '30%', 'High': '60%'},
    'Labour Strikes': {'None': '0%', 'Low': '15%', 'Medium': '35%', 'High': '55%'},
    'Custom Disruption': {'None': '0%', 'Low': '15%', 'Medium': '35%', 'High': '55%'}
}

# Setup sliders for each scenario
impact_levels = {}
for scenario, options in percentages.items():
    # Create an option label mapping for the slider
    options_with_labels = {key: f"{key} ({val})" for key, val in options.items()}
    selected_option = st.select_slider(
        label=scenario,
        options=list(options_with_labels.values()),  # Display 'None (0%)', 'Low (5%)', 'Medium (20%)', etc.
        value=options_with_labels['None'] #Default value to 'None (0%)
    )
    # Reverse lookup to get the key from the label for percentage extraction
    impact_levels[scenario] = next(key for key, value in options_with_labels.items() if value == selected_option)

# Button to predict
if st.button('Predict'):
    # Assuming placeholder functions and variables for model inputs and predictions
    model_input = prepare_model_input(carrier, original_port_of_loading, final_port_of_discharge, eta, etd)
    predicted_transit_time_days = model.predict([model_input])[0]

    # Calculate total impact percentage
    total_impact_percentage = sum(float(percentages[scenario][impact_levels[scenario]].strip('%')) for scenario in impact_levels)
    impact_multiplier = (100 + total_impact_percentage) / 100
    adjusted_transit_time_days = round(predicted_transit_time_days * impact_multiplier)

    # Calculating the predicted arrival date
    predicted_arrival_date = etd + timedelta(days=int(adjusted_transit_time_days))
    st.write(f'Predicted Arrival Date: {predicted_arrival_date.strftime("%Y-%m-%d")}')



Writing app.py


In [ ]:
!streamlit run app.py &>/content/logs.txt & npx localtunnel --port 8501 & wget -q -O - ipv4.icanhazip.com

34.145.199.146
npx: installed 22 in 6.884s
your url is: https://cuddly-flowers-sort.loca.lt


In [ ]:

# # Define the percentage mapping for each scenario
# percentages = {
#     'Red Sea disruption': {'Low': '5%', 'Medium': '20%', 'High': '40%'},
#     'COVID scenario': {'Low': '10%', 'Medium': '30%', 'High': '60%'},
#     'Port congestion': {'Low': '15%', 'Medium': '35%', 'High': '55%'},
#     'Weather Condition': {'Low': '5%', 'Medium': '20%', 'High': '40%'},
#     'War/Terrorism': {'Low': '10%', 'Medium': '30%', 'High': '60%'},
#     'Labour Strikes': {'Low': '15%', 'Medium': '35%', 'High': '55%'}
# }
# impact_levels = {}
# for scenario, options in percentages.items():
#     options_with_percents = {key: f"{key} ({val})" for key, val in options.items()}
#     selected_option = st.select_slider(
#         scenario,
#         options=list(options_with_percents.values()),  # Display 'Low (5%)', 'Medium (20%)', etc.
#         value=options_with_percents['Low']  # Default display value
#     )
#     # Reverse lookup to get the key from the value for percentage extraction
#     key_for_percent = next(key for key, value in options_with_percents.items() if value == selected_option)
#     impact_levels[scenario] = options[key_for_percent]


# # Create sliders for each scenario and collect the selected options
# # impact_levels = {}
# # for scenario, options in percentages.items():
# #     impact_levels[scenario] = st.select_slider(
# #         scenario,
# #         options=['Low', 'Medium', 'High'],
# #         value='Low'  # Default value
# #     )

# # When a button is pressed to predict
# if st.button('Predict'):
#     # Preparing model input as before
#     model_input = prepare_model_input(carrier, original_port_of_loading, final_port_of_discharge, eta, etd)

#     # Getting the prediction from the model. Assuming the model predicts transit time in days
#     predicted_transit_time_days = model.predict([model_input])[0]  # Assuming the output is a single value array

#     # Initialize a multiplier for the predicted transit time based on the selected impact levels
#     total_impact_percentage = 0

#     # # Sum the percentages of the selected impact levels
#     # for scenario, level in impact_levels.items():
#     #   percentage_value = float(percentages[scenario][level].replace('%', ''))
#     #   total_impact_percentage += percentage_value
#     total_impact_percentage = sum(float(impact_levels[scenario].replace('%', '')) for scenario in impact_levels)
#     # Calculate the adjusted transit time by adding the impact percentage
#     impact_multiplier = (100 + total_impact_percentage) / 100
#     adjusted_transit_time_days = predicted_transit_time_days * impact_multiplier


#     # Calculating the predicted arrival date by adding the transit time to ETD
#     predicted_arrival_date = etd + timedelta(days=adjusted_transit_time_days)

#     # Display the prediction
#     st.write(f'Predicted Arrival Date: {predicted_arrival_date.strftime("%Y-%m-%d")}')

In [ ]:
# %%writefile app1.py
# import streamlit as st
# import pandas as pd
# import numpy as np
# import joblib
# from datetime import datetime, timedelta
# import pydeck as pdk
# from PIL import Image

# # Load the dataset with port coordinates
# ports_coords = pd.read_excel('/content/drive/MyDrive/port_coordinates.xlsx')

# # Function to get coordinates by port code
# def get_port_coords(port_code):
#     row = ports_coords[ports_coords['Port Code'] == port_code].iloc[0]
#     return [row['Longitude'], row['Latitude']]

# # Load precomputed data
# chinese_holidays = pd.read_csv('/content/drive/MyDrive/chinese_holidays_complete.csv', sep=',', encoding='utf-8')
# chinese_holidays['date'] = pd.to_datetime(chinese_holidays['date'])
# new_df = pd.read_csv('/content/drive/MyDrive/new_df.csv')

# # Load images using PIL
# school_logo_path = '/content/drive/MyDrive/W._P._Carey_School_of_Business_logo.jpeg'
# company_logo_path = '/content/drive/MyDrive/APLL_logo.jpg'
# school_logo = Image.open(school_logo_path)
# company_logo = Image.open(company_logo_path)

# # Function to resize image
# def resize_image(image, target_width, target_height):
#     original_width, original_height = image.size
#     ratio = min(target_width/original_width, target_height/original_height)
#     new_width = int(original_width * ratio)
#     new_height = int(original_height * ratio)
#     resized_image = image.resize((new_width, new_height), Image.ANTIALIAS)
#     return resized_image

# # Resizing images
# target_width, target_height = 300, 80
# school_logo_resized = resize_image(school_logo, target_width, target_height)
# company_logo_resized = resize_image(company_logo, target_width, target_height)

# # Streamlit app setup
# st.markdown("<h1 style='text-align: center; color: white; font-size: 18px;'>Predicting Shipping Time/Transit Time to show ETA/ETD for shipments using ML Models</h1>", unsafe_allow_html=True)
# banner_container = st.container()
# with banner_container:
#     col1, col2 = st.columns([1, 1])
#     with col1:
#         st.image(school_logo_resized, use_column_width=True)
#     with col2:
#         st.image(company_logo_resized, use_column_width=True)

# left_column, right_column = st.columns([1, 3])

# with left_column:
#     model = joblib.load('/content/gradientboost_depart_new.joblib')
#     original_port_options = new_df['PORT_OF_LOAD'].unique()
#     destination_location_options = ['US','NL','CA','BE','CN']

#     carrier = st.selectbox('Carrier', new_df['CARRIER'].unique())
#     original_port_of_loading = st.selectbox('Original Port of Loading', original_port_options)
#     filtered_ports = new_df[new_df['PORT_OF_LOAD'] == original_port_of_loading]['PORT_OF_DISCHARGE'].unique()
#     final_port_of_discharge = st.selectbox('Final Port of Discharge', filtered_ports)
#     destination_location = st.selectbox('Destination Location', destination_location_options)
#     eta = st.date_input('Scheduled Time of Arrival')
#     etd = st.date_input('Scheduled Time of Departure')
#     etd = pd.to_datetime(etd)

#     if st.button('Predict'):
#         model_input = [carrier, original_port_of_loading, final_port_of_discharge, eta, etd]
#         predicted_transit_time_days = model.predict([model_input])[0]
#         predicted_arrival_date = etd + timedelta(days=predicted_transit_time_days)
#         st.write(f'Predicted Arrival Date: {predicted_arrival_date.strftime("%Y-%m-%d")}')

# with right_column:
#     original_port_coords = get_port_coords(original_port_of_loading)
#     final_port_coords = get_port_coords(final_port_of_discharge)
#     routes = pdk.Layer(
#         'LineLayer',
#         data=[{'source': original_port_coords, 'target': final_port_coords}],
#         get_source_position='source',
#         get_target_position='target',
#         get_color=[255, 0, 0],
#         get_width=2,
#         pickable=True,
#     )
#     view_state = pdk.ViewState(
#         latitude=np.mean([original_port_coords[1], final_port_coords[1]]),
#         longitude=np.mean([original_port_coords[0], final_port_coords[0]]),
#         zoom=1,
#     )
#     r = pdk.Deck(
#         layers=[routes],
#         initial_view_state=view_state
#     )
#     st.pydeck_chart(r)


Overwriting app1.py


In [ ]:
# !streamlit run app1.py &>/content/logs.txt & npx localtunnel --port 8501 & wget -q -O - ipv4.icanhazip.com

34.106.102.180
npx: installed 22 in 2.998s
your url is: https://new-olives-look.loca.lt
